In [ ]:
!pip install datasets

In [ ]:
!pip install transformers

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.7 MB/s eta 0:00:00


In [ ]:
from typing import List, Tuple

In [ ]:
from datasets import load_dataset, Dataset
import evaluate
import numpy as np
from sklearn.metrics import classification_report
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
from transformers import TrainingArguments, Trainer
from transformers import pipeline

In [ ]:
MODEL_NAME = 'ai-forever/ru-en-RoSBERTa'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/958 [00:00<?, ?B/s]

In [ ]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

In [ ]:
import random

AUGMENTATION_FACTORS = {
    'entertainment': 4,
    'geography': 2,
    'health': 2,
    'politics': 2,
    'science/technology': 2,
    'sports': 2,
    'travel': 3,
}
USE_AUGMENTATION = True

import random

def simple_augmentation(text, methods=['delete', 'swap']):
    """Простая аугментация текста"""
    words = text.split()
    if len(words) < 3:
        return text

    augmented = words.copy()
    method = random.choice(methods)

    try:
        if method == 'delete' and len(words) > 1:
            # Удаление случайного слова
            idx = random.randint(0, len(words)-1)
            augmented.pop(idx)

        elif method == 'swap' and len(words) > 2:
            # Обмен местами двух соседних слов
            idx = random.randint(0, len(words)-2)
            augmented[idx], augmented[idx+1] = augmented[idx+1], augmented[idx]
    except:
        pass

    return ' '.join(augmented)

def augment_dataset_simple(dataset, augmentation_factor=2, filter='None'):
    """Простая аугментация датасета"""
    augmented_data = []

    for item in dataset:
        # Добавляем оригинальный пример
        augmented_data.append(item)

        # Добавляем аугментированные примеры
        for i in range(AUGMENTATION_FACTORS.get(item['category'])):
            augmented_item = item.copy()
            augmented_item['text'] = simple_augmentation(item['text'])
            augmented_item['index_id'] = item['index_id'] + len(dataset) * i
            augmented_item['category'] = item['category']
            augmented_data.append(augmented_item)

    return Dataset.from_list(augmented_data)

In [ ]:
if USE_AUGMENTATION:
    print("Инициализация аугментатора...")

    print("Аугментация обучающей выборки...")
    train_set = augment_dataset_simple(train_set, AUGMENTATION_FACTORS)
else:
    print("Аугментация отключена")

Инициализация аугментатора...
Аугментация обучающей выборки...


In [ ]:
print(train_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 2371
})


In [ ]:
MINIBATCH_SIZE = 32
tokenized_train_set = train_set.map(
    lambda it: tokenizer(it['text'], truncation=True),
    batched=True, batch_size=MINIBATCH_SIZE
)
tokenized_validation_set = validation_set.map(
    lambda it: tokenizer(it['text'], truncation=True),
    batched=True, batch_size=MINIBATCH_SIZE
)

Map:   0%|          | 0/2371 [00:00<?, ? examples/s]

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_train_set)

Dataset({
    features: ['index_id', 'category', 'text', 'input_ids', 'attention_mask'],
    num_rows: 2371
})


In [ ]:
cls_metric = evaluate.load('f1')

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return cls_metric.compute(predictions=predictions, references=labels, average='macro')

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
indices_of_categories = list(range(len(list_of_categories)))
n_categories = len(list_of_categories)
print(f'Categories for classification are: {list_of_categories}')
id2label = dict(zip(indices_of_categories, list_of_categories))
label2id = dict(zip(list_of_categories, indices_of_categories))

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [ ]:
labeled_train_set = tokenized_train_set.add_column(
    'label',
    [label2id[val] for val in tokenized_train_set['category']]
)
labeled_validation_set = tokenized_validation_set.add_column(
    'label',
    [label2id[val] for val in tokenized_validation_set['category']]
)

In [ ]:
print(labeled_train_set)

In [ ]:
classifier = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=n_categories, id2label=id2label, label2id=label2id
).cuda()
for param in classifier.parameters(): param.data = param.data.contiguous()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir='rubert_sib200',
    learning_rate=2e-5,
    per_device_train_batch_size=MINIBATCH_SIZE,
    per_device_eval_batch_size=MINIBATCH_SIZE,
    num_train_epochs=8,
    weight_decay=1e-1,
    eval_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    warmup_ratio=0.1,
    fp16=True,
)

In [ ]:
trainer = Trainer(
    model=classifier,
    args=training_args,
    train_dataset=labeled_train_set,
    eval_dataset=labeled_validation_set,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-1253556369.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train()

In [ ]:
classifiсation_pipeline = pipeline('text-classification', model=classifier, tokenizer=tokenizer, device=0)

Device set to use cuda:0


In [ ]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(list(validation_set['text']))))
y_true = validation_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       0.67      0.67      0.67         9
         geography       0.80      1.00      0.89         8
            health       1.00      0.82      0.90        11
          politics       0.93      0.93      0.93        14
science/technology       0.96      0.96      0.96        25
            sports       0.92      0.92      0.92        12
            travel       0.75      0.75      0.75        20

          accuracy                           0.87        99
         macro avg       0.86      0.86      0.86        99
      weighted avg       0.87      0.87      0.87        99



In [ ]:
y_pred = list(map(lambda x: x['label'], classifiсation_pipeline(list(test_set['text']))))
y_true = test_set['category']
print(classification_report(y_true=y_true, y_pred=y_pred))

                    precision    recall  f1-score   support

     entertainment       0.88      0.74      0.80        19
         geography       0.94      0.94      0.94        17
            health       0.91      0.91      0.91        22
          politics       0.97      0.93      0.95        30
science/technology       0.89      0.94      0.91        51
            sports       0.85      0.88      0.86        25
            travel       0.93      0.93      0.93        40

          accuracy                           0.91       204
         macro avg       0.91      0.90      0.90       204
      weighted avg       0.91      0.91      0.91       204

